# Publications Hypergraph Demo Notebook

This notebook is used to replcate the use case experiment in the paper **PAPER LINK HERE** that demonstrates the use of the hypergraph interchange formation (HIF).

The following hypergraph libraries have the HIF loading and saving functions allowing for the analysis of the same hypergraph over each library:
* HyperNetX (HNX)
* HyperGraphX (HGX)
* SimpleHypergraphs
* TopoNetX
* Hypergraph Analysis Toolbox (HAT)
* compleX Group Interactions (XGI)

## Publications Data as a Hypergraph

This dataset consists of open source publications with the keyword "Hypergraph" and was collected from ArXiv, Biorxiv, DBLP and Osti. The Hypergraph has hyperedges as publications and nodes as authors. The hyperedges have attributes including funding agencies, abstract, publication date, tags, and source, while the nodes have the attributes of institutions.

In [ ]:
# import standard packages for analysis and plotting
import json

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pylab as pylab


def set_fonts(extra_params={}):
    params = {
        "font.family": "Serif",
        # "font.sans-serif": ["Tahoma", "DejaVu Sans", "Lucida Grande", "Verdana"],
        "mathtext.fontset": "cm",
        "legend.fontsize": 14,
        "axes.labelsize": 14,
        "axes.titlesize": 14,
        "xtick.labelsize": 14,
        "ytick.labelsize": 14,
        "figure.titlesize": 14,
    }
    for key, value in extra_params.items():
        params[key] = value
    pylab.rcParams.update(params)

set_fonts()

In [ ]:
file_path = "data/publications.hif.json"
with open(file_path, "r") as f:
    HIF_publications = json.load(f)

## HAT

In [ ]:
from HAT import Hypergraph as HatHypergraph
from HAT.metrics import nonlinear_eigenvector_centrality

H_hat = HatHypergraph.from_hif(HIF_publications)
node_centrality, edge_centrality = nonlinear_eigenvector_centrality(H_hat, model='Linear')
i, e = max(enumerate(edge_centrality), key=lambda y: y[1])
name = H_hat.edges["edge"][i]

plt.figure(figsize=(5.5, 4))
plt.subplot(211)
plt.hist(node_centrality, bins=30)
plt.yscale("log")
plt.xlabel("Node centrality")
plt.ylabel("Number")


plt.subplot(212)
plt.hist(edge_centrality, bins=30)
plt.yscale("log")
plt.xlabel("Edge centrality")
plt.ylabel("Number")
plt.tight_layout()
sns.despine()
plt.savefig("../figures/hat.svg", dpi=1000)

## Hypergraphx

## HyperNetX

Here we demo loading in the hypergraph using HNX and demoing the unique homology capabilities of the library.

In [ ]:
import hypernetx as hnx

In [ ]:
H_hnx = hnx.from_hif(HIF_publications)

In [ ]:
plt.figure(figsize=(5.5, 5.5))
hnx.draw(
    H_hnx,
    with_edge_labels=False,
    with_node_labels=False,
    node_radius=0.2,
    edges_kwargs={"lw": 0.5},
)
plt.tight_layout()
plt.savefig("../figures/full_publications.pdf", dpi=1000)

In [ ]:
# Getting the main connected component of hypergraph
import numpy as np

Hs = list(H_hnx.s_component_subgraphs(s=1, return_singletons=False))
I = np.argsort([len(H_CC.incidences.dataframe) for H_CC in Hs])
Hs = [Hs[i] for i in I]
H = Hs[-1]

In [ ]:
pos = hnx.draw(
    H,
    with_edge_labels=False,
    with_node_labels=True,
    node_radius=0.2,
    edges_kwargs={"lw": 0.5},
    return_pos=True,
)
plt.show()

## One-Dimensional Homology of a Hypergraph Using HNX

In [ ]:
import hypernetx as hnx
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# Assuming your hypergraph 'H' is already defined
s = 1
edge_centrality = hnx.s_closeness_centrality(H, s=s)

# Get the centrality values and the edges
edges = list(H.edges)
centrality_values = np.array([edge_centrality[e] for e in edges])

# Normalize the centrality values to the range [0, 1]
if centrality_values.size > 0:
    normalized_centrality = (centrality_values - np.min(centrality_values)) / (
        np.max(centrality_values) - np.min(centrality_values)
    )
else:
    normalized_centrality = np.array([])  # Handle the case of no edges

# Choose a colormap
cmap = cm.get_cmap("viridis")

# Map the normalized centrality values to colors
edge_colors = cmap(normalized_centrality)

# Create a dictionary to pass to edges_kwargs for coloring
edge_style = {}
for i, edge in enumerate(edges):
    if i < len(edge_colors):
        edge_style[edge] = edge_colors[i]

# Create the hypergraph plot
fig, ax = plt.subplots(figsize=(5.5, 4))
hnx.draw(
    H,
    pos=pos,
    with_edge_labels=False,
    with_node_labels=False,
    node_radius=0.5,
    edges_kwargs={"color": list(edge_style.values()), "lw": 2, "alpha": 0.5},
    ax=ax,
)

# Add the colorbar
sm = cm.ScalarMappable(
    cmap=cmap,
    norm=plt.Normalize(vmin=np.min(centrality_values), vmax=np.max(centrality_values)),
)
sm.set_array([])  # For older versions of matplotlib
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label(f"{s}-Closeness Centrality", fontsize=14)
cbar.set_ticks([0.25, 0.35, 0.45, 0.55])
plt.tight_layout()
plt.savefig("../figures/hnx.svg", dpi=1000)
plt.show()

In [ ]:
s = 1
print(f"The {s}-closeness centrality of the edges in hypergraph H is:")
print("-" * 100)

d = hnx.s_closeness_centrality(H, s=s)
for e in H.edges:
    print(f"{e}: {d[e]}")

In [ ]:
from julia.api import Julia
import julia
# julia.install()
jl = Julia(compiled_modules=False)

%load_ext julia.magic

In [ ]:
%%julia
using Pkg
Pkg.add(url="https://github.com/AleksanderWWW/SimpleHypergraphs.jl.git", rev="aw/hif-import-export", io=devnull)
Pkg.add("Test", io=devnull)
Pkg.add("StatsBase", io=devnull)
Pkg.add("Random", io=devnull)
Pkg.add("DataStructures", io=devnull)
Pkg.add("Graphs", io=devnull)
Pkg.add("GraphPlot", io=devnull)
Pkg.add("Colors", io=devnull)
Pkg.add("ColorSchemes", io=devnull)
Pkg.add("Compose", io=devnull)
using Test, SimpleHypergraphs, StatsBase
using Random
using DataStructures
using Graphs
using GraphPlot
using Colors, ColorSchemes

In [ ]:
%%julia
file_path = "data/publications.hif.json" # Your specified file path
hg = hg_load(file_path, HIF_Format(), T=Bool)

cmpts = get_connected_components(hg)
n, id = findmax(length, cmpts)

to_select = Set(cmpts[id])


for v in nhv(hg):-1:1
    if !(v in to_select)
        remove_vertex!(hg, v)
    end
end

SimpleHypergraphs.prune_hypergraph(hg)

cnm = CFModularityCNMLike(5000)
Random.seed!(1234)
cnm_comms = findcommunities(hg, cnm)

println("We found $(length(cnm_comms.bp)) communities ")

t = Graphs.Graph(get_twosection_adjacency_mx(hg;replace_weights=1))

my_colors = vcat(ColorSchemes.rainbow[range(1, stop=length(ColorSchemes.rainbow), step=3)], ColorSchemes.rainbow[2]);
function get_color(i, comms, colors)
    for j in 1:length(comms)
        if length(comms[j]) > 1 && i in comms[j]
            return "#"*hex(colors[j % length(colors) + 1])
        end
    end
    return "#000000"
end;

degrees = Graphs.degree.(Ref(t), Graphs.vertices(t));
dsize = 3 .+ 1.5 .* degrees./maximum(degrees);

saveplot(
    gplot(
        t,
        nodesize = dsize,
        nodefillc=get_color.(1:Graphs.nv(t), Ref(cnm_comms.bp), Ref(reverse(my_colors))),
        plot_size = (14cm, 10cm)
    ),
    "../figures/simplehypergraphs.svg"
)

# XGI

In [ ]:
import xgi

In [ ]:
H_xgi = xgi.read_hif(file_path)
testH = H_xgi.cleanup(in_place=False)

In [ ]:
fig = plt.figure(figsize=(5.5, 4))
ax = plt.gca()
# ax1 = ax.inset_axes([0, 0, 0.7, 1])
# ax1.set_axis_off()
xgi.draw(
    testH,
    # ax=ax1,
    pos=xgi.pca_transform(xgi.pairwise_spring_layout(testH, seed=0)),
    hull=True,
    node_size=testH.nodes.degree,
    node_fc=testH.nodes.clique_eigenvector_centrality,
)

ax1 = ax.inset_axes([0.7, 0.25, 0.25, 0.125])
y1 = testH.nodes.degree.asnumpy()
y2 = testH.nodes.average_neighbor_degree.asnumpy()
idx = np.argsort(y1)[::-1]
ax1.plot(y1[idx], "-", color="maroon")
ax1.plot(y2[idx], "--", color="teal")
ax1.set_xticks([], [])
ax1.set_ylabel("Deg.", labelpad=12)
ax1.set_yscale("log")
sns.despine(ax=ax1)

ax2 = ax.inset_axes([0.7, 0.0, 0.25, 0.125])
sns.despine(ax=ax2)
y1 = testH.nodes.clique_eigenvector_centrality.asnumpy()
y2 = testH.nodes.h_eigenvector_centrality.asnumpy()
idx = np.argsort(y1)[::-1]
ax2.plot(y1[idx], "-", color="maroon")
ax2.plot(y2[idx], "--", color="teal")
ax2.set_xlabel("Node", labelpad=10)
ax2.set_ylabel("Cent.")
ax2.set_yscale("log")
plt.tight_layout()
plt.savefig("../figures/xgi.svg", dpi=1000)